In [1]:
from collections import defaultdict
import pytrec_eval
from statistics import mean
import scipy.stats as stats

combine_type = 'none'

VALIDATION_METRIC = 'ndcg_cut_10'     #'recip_rank' #'ndcg_cut_10' 
top_k = 10
for k1 in [60]:
    for k2 in [60]:
        for yr in ['19','20']:
        
            qrel_file = f"../msmarco/20{yr}qrels-pass.txt" #"../msmarco/qrels.dev.tsv"  f"../msmarco/20{yr}qrels-pass.txt"
            qrels = defaultdict(dict)
            with open(qrel_file) as f:
                for line in f:
                    try:
                        qid, _, did, rel = line.strip().split("\t")
                    except:
                        qid, _, did, rel = line.strip().split(" ")
                    if int(rel) > 0:
                        qrels[qid][did] = int(rel)

            run = defaultdict(dict)
          
            with open(f"../msmarco/index_splade_num1_marginmse.psg.{yr}.trec.trec") as f: #f"../msmarco/wentai_splade_20{yr}.trec", "../msmarco/wentai_splade_dev_top1000.trec"  f"../msmarco/splade-remove/test{yr}.remove60.1000"
                for line in f:
                    qid, _, did, rank, _, _ = line.split("\t")
                    run[qid][did] = 1.0/(int(rank) + k1)
            
            run_retriever = {q: list(run[q].items()) for q in run}
            
            file_name = f"results/output_{yr}_spladefromnum1marginmse_colbertnum3_spladeneg_1e-6_100000.tsv"
            if combine_type == "weight":
                sub_run_prf = defaultdict(dict)
                with open(file_name + ".prf") as f: 
                    for line in f:
                        qid, did, score = line.split("\t")
                        sub_run_prf[qid][did] = float(score)
                       
                sub_run = defaultdict(list)
                with open(file_name) as f:
                    for line in f:
                        qid, did, score = line.split("\t")
                        sub_run[qid].append((did, float(score) + 0.1 * sub_run_prf[qid][did]))
                for q in sub_run:
                    sub_run[q] = sorted(sub_run[q], key=lambda x: -x[1])


            elif combine_type in ["none", "rrf"]:
                sub_run = defaultdict(list)
                with open(file_name) as f: #f"output_{yr}_colspla_from_colbert_3e-6_negpersys20_200k.run.json.tsv"
                    for line in f:
                        qid, did, score = line.split("\t")
                        sub_run[qid].append((did, float(score)))

                for q in sub_run:
                    sub_run[q] = sorted(sub_run[q], key=lambda x: -x[1])
                    
                if combine_type == "rrf":
                    sub_run_prf = defaultdict(list)
                    with open(file_name + ".prf") as f: #f"output_{yr}_colspla_from_colbert_3e-6_negpersys20_200k.run.json.tsv"
                        for line in f:
                            qid, did, score = line.split("\t")
                            sub_run_prf[qid].append((did, float(score)))

                    for q in sub_run_prf:
                        sub_run_prf[q] = sorted(sub_run_prf[q], key=lambda x: -x[1])

            
            for q in sub_run:
                x1 = [x[0] for x in run_retriever[q]]
                x2 = [x[0] for x in sub_run[q]]
                tau, p_value = stats.kendalltau(x1[:10], x2[:10])

                for idx, item in enumerate(sub_run[q]):
                    if item[0] in run[q]:
                        run[q][item[0]] += 1.0/(int(idx+1) + k2)
                    else:
                        run[q][item[0]] = 1.0/(int(idx+1) + k2)

                if combine_type in ["none", "weight"]:
                    threshold = -1
                else:
                    threshold = 1
                if tau < threshold:
                    k3 = 240
                    for idx, item in enumerate(sub_run_prf[q]):
                        if item[0] in run[q]:
                            run[q][item[0]] += 1.0/(int(idx+1) + k3)
                        else:
                            run[q][item[0]] = 1.0/(int(idx+1) + k3)
                else:
                    continue
                
            
            trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
            eval_scores = trec_eval.evaluate(run)
            print(k1,k2, yr, mean([d[VALIDATION_METRIC] for d in eval_scores.values()]))

60 60 19 0.7455742153911629
60 60 20 0.7448402226925842


In [2]:
run_retriever[q]

[('2667345', 0.01639344262295082),
 ('7259157', 0.016129032258064516),
 ('61071', 0.015873015873015872),
 ('5118373', 0.015625),
 ('2667347', 0.015384615384615385),
 ('61068', 0.015151515151515152),
 ('4800890', 0.014925373134328358),
 ('2667353', 0.014705882352941176),
 ('5300651', 0.014492753623188406),
 ('8219249', 0.014285714285714285),
 ('5831708', 0.014084507042253521),
 ('2667350', 0.013888888888888888),
 ('4696636', 0.0136986301369863),
 ('8772071', 0.013513513513513514),
 ('4696640', 0.013333333333333334),
 ('5300654', 0.013157894736842105),
 ('2667352', 0.012987012987012988),
 ('4664997', 0.01282051282051282),
 ('5118376', 0.012658227848101266),
 ('8772075', 0.0125),
 ('4665002', 0.012345679012345678),
 ('1335718', 0.012195121951219513),
 ('2699754', 0.012048192771084338),
 ('6693958', 0.011904761904761904),
 ('2613393', 0.011764705882352941),
 ('8772074', 0.011627906976744186),
 ('7282176', 0.011494252873563218),
 ('61062', 0.011363636363636364),
 ('3706456', 0.0112359550561

In [3]:
sub_run[q]

[('2667347', 20.48802947998047),
 ('61068', 20.149991989135742),
 ('2667345', 19.95679473876953),
 ('4696640', 19.88412857055664),
 ('8219249', 19.84771728515625),
 ('7259157', 19.623445510864258),
 ('61071', 19.623445510864258),
 ('5118373', 19.603511810302734),
 ('2667353', 19.5698299407959),
 ('4800890', 19.299964904785156),
 ('2667350', 19.18178939819336),
 ('5300651', 19.17664337158203),
 ('4696636', 19.078563690185547),
 ('5300654', 18.84521484375),
 ('5831708', 18.822267532348633),
 ('7282176', 18.695932388305664),
 ('4800886', 18.67310333251953),
 ('8772071', 18.585079193115234),
 ('8772074', 18.419469833374023),
 ('4664997', 18.412338256835938),
 ('5118376', 18.373817443847656),
 ('8772075', 18.33997344970703),
 ('5118378', 18.159687042236328),
 ('1335718', 18.02716636657715),
 ('2613393', 17.96300506591797),
 ('4665002', 17.957786560058594),
 ('5831711', 17.93838119506836),
 ('7282174', 17.884572982788086),
 ('61062', 17.882465362548828),
 ('4665000', 17.879735946655273),
 ('

In [43]:
for q in sub_run:
    sub_run[q] = dict(sub_run[q])

In [5]:
def g(idx, k = 60):
     return 1.0/(int(idx+1) + k)

In [8]:
results = sorted([items for items in run[q].items()], key = lambda x: -x[1])

In [36]:
arr1 = run_retriever[q]
def get_topk_safe(arr1, k=10):
    K = len(arr1)
    arr_dict1 = dict(arr1)
    mink = 0
    evaluated = []
    for item in arr1:
        new_max = g(0) + item[1]
        if new_max <= mink:
            break
        else:
            rr_score = sub_run[q][item[0]]
            evaluated.append([item[0], rr_score])
            order = [x[0] for x in sorted(evaluated, key=lambda x: -x[1])]
            mink = []
            for i, x in enumerate(evaluated):
                cur_score = g(K - len(evaluated) + order.index(x[0])) + g(i)
                mink.append(cur_score)
            if len(mink) < k:
                mink = 0
            else:
                mink = sorted(mink, reverse=True)[k-1]

    mink= []           
    print(len(evaluated))
    for idx, item in enumerate(sorted(evaluated, key = lambda x: -x[1])):
        mink.append([item[0], g(idx) + arr_dict1[item[0]]])

    topk = sorted(mink, key=lambda x: -x[1])[:k]
    return dict(topk), len(evaluated)

In [26]:
bench=dict(sorted([x for x in run[q].items()], key = lambda x: -x[1])[:k])

In [28]:
topk, num = get_topk_safe(run_retriever[q])
topk == bench

764


True

In [37]:
arr1 = run_retriever[q]

def get_topk_fast(arr1, k=10):
    K = len(arr1)
    mink = 0
    arr_dict1 = dict(arr1)
    evaluated = []
    for item in arr1:
        new_max = g(0) + item[1]
        if new_max <= mink:
            break
        else:
            rr_score = sub_run[q][item[0]]
            evaluated.append([item[0], rr_score])
            order = [x[0] for x in sorted(evaluated, key=lambda x: -x[1])]
            mink = []
            for i, x in enumerate(evaluated):
                cur_score = g(order.index(x[0])) + g(i)
                mink.append(cur_score)
            if len(mink) < k:
                mink = 0
            else:
                mink = sorted(mink, reverse=True)[k-1]
    mink= []           
    print(len(evaluated))
    for idx, item in enumerate(sorted(evaluated, key = lambda x: -x[1])):
        mink.append([item[0], g(idx) + arr_dict1[item[0]]])

    topk = sorted(mink, key=lambda x: -x[1])[:k]
    return dict(topk), len(evaluated)

In [30]:
topk, num = get_topk_fast(run_retriever[q])
topk == bench

19


True

In [38]:
arr1 = run_retriever[q]

def get_topk_even(arr1, k=10):
    K = len(arr1)
    mink = 0
    evaluated = []
    arr_dict1 = dict(arr1)
    for item in arr1:
        new_max = g(0) + item[1]
        if new_max <= mink:
            break
        else:
            rr_score = sub_run[q][item[0]]
            evaluated.append([item[0], rr_score])
            order = [x[0] for x in sorted(evaluated, key=lambda x: -x[1])]
            mink = []
            for i, x in enumerate(evaluated):
                cur_score = g(int(order.index(x[0])/len(evaluated) * K)) + g(i)
                mink.append(cur_score)
            if len(mink) < k:
                mink = 0
            else:
                mink = sorted(mink, reverse=True)[k-1]
    len(evaluated)

    mink= []           
    print(len(evaluated))
    for idx, item in enumerate(sorted(evaluated, key = lambda x: -x[1])):
        mink.append([item[0], g(idx) + arr_dict1[item[0]]])

    topk = sorted(mink, key=lambda x: -x[1])[:k]
    return dict(topk), len(evaluated)

In [33]:
topk, num = get_topk_even(run_retriever[q])
topk == bench

114


True

In [40]:
safe = []
even = []
fast = []
for q in run_retriever:
    bench=dict(sorted([x for x in run[q].items()], key = lambda x: -x[1])[:k])
    topk, num = get_topk_even(run_retriever[q])
    even.append([num, topk == bench])
    
    topk, num = get_topk_fast(run_retriever[q])
    fast.append([num, topk == bench])
    
    topk, num = get_topk_safe(run_retriever[q])
    safe.append([num, topk == bench])

136
26
766
132
23
770
135
23
778
118
23
768
146
36
799
130
24
770
127
23
770
124
22
769
121
20
768
130
22
789
129
22
783
124
22
769
121
21
787
135
23
765
153
24
775
120
22
777
124
22
769
135
20
772
124
22
769
135
24
772
130
21
765
124
20
769
114
20
764
118
19
766
124
22
769
124
23
770
126
21
775
127
23
771
120
20
765
124
20
765
137
21
785
134
24
773
124
22
769
126
26
773
124
22
769
126
24
772
132
23
770
124
21
769
135
23
768
124
22
769
118
19
766
118
23
770
146
27
783
171
19
780
132
23
778
120
22
771
132
21
770
125
20
766
117
21
768
132
23
770
135
27
780
121
20
767
114
19
764


TypeError: list indices must be integers or slices, not str

In [3]:
from collections import defaultdict
import pytrec_eval
from statistics import mean
import scipy.stats as stats

combine_type = 'none'

VALIDATION_METRIC = 'ndcg_cut_10'     #'recip_rank' #'ndcg_cut_10' 
top_k = 10

for yr in ['19','20']:

    qrel_file = f"../msmarco/20{yr}qrels-pass.txt" #"../msmarco/qrels.dev.tsv"  f"../msmarco/20{yr}qrels-pass.txt"
    qrels = defaultdict(dict)
    with open(qrel_file) as f:
        for line in f:
            try:
                qid, _, did, rel = line.strip().split("\t")
            except:
                qid, _, did, rel = line.strip().split(" ")
            if int(rel) > 0:
                qrels[qid][did] = int(rel)

    run = defaultdict(dict)
    #f"../msmarco/index_splade_num1_marginmse.psg.{yr}.trec.trec"
    with open(f"../msmarco/splade_klfr_5-1_num1.20{yr}.trec.trec") as f: #f"../msmarco/wentai_splade_20{yr}.trec",   f"../msmarco/splade-remove/test{yr}.remove60.1000"  f"../msmarco/yifan_bm25_top1000.psg.20{yr}.trec.trec"
        for line in f:
            qid, _, did, rank, _, _ = line.split("\t")
            run[qid][did] = -float(rank)
    
    run_retriever = {q: list(run[q].items()) for q in run}
    
    
    trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
    eval_scores = trec_eval.evaluate(run)
    print(yr, mean([d[VALIDATION_METRIC] for d in eval_scores.values()]))


19 0.7444273585489548
20 0.7287996194997873


In [6]:
from collections import defaultdict
import pytrec_eval
from statistics import mean
import scipy.stats as stats

combine_type = 'none'

VALIDATION_METRIC = 'recip_rank'     #'recip_rank' #'ndcg_cut_10' 
top_k = 10

for yr in ['dev']:

    qrel_file = f"../msmarco/qrels.dev.tsv" #"../msmarco/qrels.dev.tsv"  f"../msmarco/20{yr}qrels-pass.txt"
    qrels = defaultdict(dict)
    with open(qrel_file) as f:
        for line in f:
            try:
                qid, _, did, rel = line.strip().split("\t")
            except:
                qid, _, did, rel = line.strip().split(" ")
            if int(rel) > 0:
                qrels[qid][did] = int(rel)

    run = defaultdict(dict)
    #f"../msmarco/index_splade_num1_marginmse.psg.{yr}.trec.trec"
    with open(f"../msmarco/splade_klfr_5-1_num1.{yr}.trec.trec") as f: #f"../msmarco/wentai_splade_20{yr}.trec",   f"../msmarco/splade-remove/test{yr}.remove60.1000"  f"../msmarco/yifan_bm25_top1000.psg.20{yr}.trec.trec"
        for line in f:
            qid, _, did, rank, _, _ = line.split("\t")
            run[qid][did] = -float(rank)
    
    top_run = defaultdict(dict)
    for q in run:
        docs = sorted(run[q].items(), key=lambda x: -x[1])
        for item in docs[:top_k]:
            top_run[q][item[0]] = item[1]
    
    
    trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
    eval_scores = trec_eval.evaluate(top_run)
    print(yr, mean([d[VALIDATION_METRIC] for d in eval_scores.values()]))


dev 0.402014031018329


# simulation - independent ordering

In [1]:
import random
arr1 = list(range(0, 100))
random.shuffle(arr1)

In [2]:
arr1

[35,
 95,
 60,
 69,
 4,
 87,
 33,
 0,
 97,
 55,
 25,
 43,
 17,
 1,
 13,
 47,
 98,
 18,
 22,
 29,
 78,
 68,
 92,
 99,
 96,
 30,
 65,
 64,
 89,
 54,
 73,
 50,
 71,
 37,
 16,
 39,
 32,
 51,
 72,
 67,
 24,
 6,
 2,
 40,
 10,
 42,
 77,
 74,
 7,
 49,
 58,
 26,
 90,
 31,
 83,
 11,
 91,
 8,
 82,
 38,
 66,
 62,
 12,
 20,
 79,
 41,
 23,
 63,
 88,
 34,
 45,
 93,
 70,
 46,
 59,
 56,
 80,
 28,
 15,
 85,
 19,
 57,
 84,
 94,
 5,
 81,
 9,
 36,
 52,
 44,
 21,
 76,
 3,
 61,
 48,
 75,
 53,
 27,
 14,
 86]

In [74]:
def get_topk_even(arr1, k=10):
    K = len(arr1)
    mink = 0
    evaluated = []
    for idx, item in enumerate(arr1):
        new_max = g(0) + g(idx)
        if new_max <= mink:
            break
        else:
            evaluated.append([idx, item])
            order = [x for x in sorted(evaluated, key=lambda x: -x[1])]
            mink = []
            for x in evaluated:
                cur_score = g(int(order.index(x)/len(evaluated) * K)) + g(x[0])
                mink.append(cur_score)
            if len(mink) < k:
                mink = 0
            else:
                mink = sorted(mink, reverse=True)[k-1]
    

    mink= []           
    for idx, x in enumerate(sorted(evaluated, key = lambda x: -x[1])):
        mink.append([x[0], g(idx) + g(x[0])])

    topk = sorted(mink, key=lambda x: -x[1])[:k]
    return dict(topk), len(evaluated)

In [86]:
get_topk_even(arr1)

({4: 0.03125763125763126,
  16: 0.028371628371628373,
  14: 0.028258706467661692,
  7: 0.02821939586645469,
  26: 0.02788769549651404,
  13: 0.027402402402402402,
  0: 0.02738245361196181,
  9: 0.02694394213381555,
  21: 0.0264808362369338,
  38: 0.02623004235907462},
 55)

In [13]:
def get_topk_safe(arr1, k=10):
    K = len(arr1)
    mink = 0
    evaluated = []
    for rt, rr in enumerate(arr1):
        new_max = g(0) + g(rt) #assume rerank is top
        if new_max <= mink:
            break
        else:
            evaluated.append([rt, rr])
            order = sorted(evaluated, key=lambda x: x[1]) #ranked based on rerank order
            mink = []
            for x in evaluated:
                cur_score = g(x[0]) + g(int(K - len(evaluated) + order.index(x))) 
                mink.append(cur_score)
            if len(mink) < k:
                mink = 0
            else:
                mink = sorted(mink, reverse=True)[k-1]
    

    topk= []           
    order = sorted(evaluated, key = lambda x: x[1])
    for idx, x in enumerate(order):
        topk.append([x[0], g(idx), g(x[0])])
    
    topk = sorted(topk, key=lambda x: -(x[1]+x[2]))[:k]
    
    return topk, len(evaluated)

In [18]:
get_topk_safe(arr1)

([[7, 0.01639344262295082, 0.014705882352941176],
  [4, 0.015625, 0.015384615384615385],
  [13, 0.016129032258064516, 0.013513513513513514],
  [0, 0.011111111111111112, 0.01639344262295082],
  [14, 0.014084507042253521, 0.013333333333333334],
  [12, 0.013513513513513514, 0.0136986301369863],
  [10, 0.012345679012345678, 0.014084507042253521],
  [6, 0.011363636363636364, 0.014925373134328358],
  [17, 0.013333333333333334, 0.01282051282051282],
  [42, 0.015873015873015872, 0.009708737864077669]],
 81)

In [16]:
k=10
bench = sorted([[x, g(y), g(x)] for x,y in enumerate(arr1)], key=lambda x: -(x[1]+x[2]))[:k]
bench

[[7, 0.01639344262295082, 0.014705882352941176],
 [4, 0.015384615384615385, 0.015384615384615385],
 [13, 0.016129032258064516, 0.013513513513513514],
 [14, 0.013513513513513514, 0.013333333333333334],
 [0, 0.010416666666666666, 0.01639344262295082],
 [12, 0.01282051282051282, 0.0136986301369863],
 [10, 0.011627906976744186, 0.014084507042253521],
 [42, 0.015873015873015872, 0.009708737864077669],
 [6, 0.010638297872340425, 0.014925373134328358],
 [17, 0.012658227848101266, 0.01282051282051282]]